# Section 2 Risk Factor Extraction

In section 1, we have downloaded the fincancial reports from 2019Q1 to 2023Q3. In this section, we are going to extract the risk factor text from each of these files. According to the requirement of the 10-K form given by SEC (https://www.sec.gov/files/form10-k.pdf), there is an section named "Item 1A. Risk Factors" included in the form  which shows the potential risks faced by the company. Therefore, it is important for us to filter and cleanse the text from this part so that we can use it for further analysis.

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import re
import html
import warnings
warnings.filterwarnings("ignore")

# 2.5 Real Estate Industry (6500, 6798)

In this section, we only focus on companies in Real Estate Industry.

## 2.4.1 Industry 6500

### 1. Read in the files and store in a dictionary

In [2]:
folderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Downloaded Financial Reports/Real Estate Industry/Industry_6500/"
fileName_lst = os.listdir(folderPath)
filePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in fileName_lst:  
    # encode each document
    filePath = str(folderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        fileText = file.read()
    filePair_dict[fileName] = fileText

In [3]:
filePair_dict.keys()

dict_keys(['0000096869-23-000006.txt', '0000315858-23-000011.txt', '0000844059-23-000014.txt', '0000894245-23-000030.txt', '0000894245-23-000031.txt', '0000894245-23-000032.txt', '0000894245-23-000034.txt', '0000950170-22-005732.txt', '0000950170-23-003074.txt', '0000950170-23-004172.txt', '0000950170-23-004385.txt', '0000950170-23-007891.txt', '0000950170-23-011406.txt', '0000950170-23-011418.txt', '0001065949-22-000050.txt', '0001138118-23-000009.txt', '0001140361-22-034559.txt', '0001213900-23-024126.txt', '0001299969-23-000010.txt', '0001408100-23-000055.txt', '0001411342-23-000056.txt', '0001437749-23-007690.txt', '0001437749-23-007848.txt', '0001456772-23-000012.txt', '0001476150-23-000004.txt', '0001477932-22-007245.txt', '0001482512-23-000048.txt', '0001493152-22-009873.txt', '0001493152-22-010033.txt', '0001493152-23-008954.txt', '0001493152-23-009286.txt', '0001493152-23-010224.txt', '0001493152-23-010372.txt', '0001515971-22-000147.txt', '0001555074-23-000012.txt', '00015583

### 2. Extract the raw risk factor text

In [4]:
def remove_tags(text):
    # replace the tag included by <> to a \ 
    pattern1 = r'<[^>]*>'
    cleanText = re.sub(pattern1, "|", text)
    
    # replace the multiple continued \ to a single \
    cleanText = re.sub(r'\s*\|+\s*', '|', cleanText)
    cleanText = re.sub(r'\|+', '|', cleanText)
    
    # replace the unicode
    cleanText = html.unescape(cleanText)
    
    # remove the line break
    cleanText = cleanText.replace('\n', '')
    # remove the pure numbers text between \
    cleanText = re.sub(r'\|(\d+)\|', '|', cleanText)
    # remove the unrecognized string
    cleanText = re.sub(r'|\xa0\|', '', cleanText)
    cleanText = re.sub(r'|\xa0|', '', cleanText)
    return cleanText

In [5]:
def separate_main_body(text):
    lastContentInTC_lst = [">form 10-k summary<", ">item 16. form 10-k summary<", r">exhibits, financial statement schedules<", 
                           r">exhibits and financial statement schedules<", r"exhibits and financial statement schedules <",
                           ">exhibits<"]
    start_idx = -1
    for content in lastContentInTC_lst:
        if content in text.lower():
            start_idx = text.lower().find(content)
            break
            
    if start_idx == -1:
        return False
    
    mainBody = text[start_idx:]
    mainBody = html.unescape(mainBody)
    return mainBody

In [6]:
def extract_risk_factors(mainBody, fileName):
    # match the risk factor
    keywordMatch = re.findall(r">item 1a.\s+risk factors\.?\<", mainBody.lower())
    
    # if fail to match the pattern, the pattern might be segmented be some html tags, so re-match
    if not keywordMatch:
        keyword = seperated_keyword_rematch(mainBody)
        
    # if match the pattern successfully, just keep the keyword of risk factor
    else:
        keyword = keywordMatch[0]
    
    # check if match successfully
    if not keyword:
        return False
    keywordCheck_lst = ["item", r"item 1a\.?", "risk", "factors", "risk factors"]
    if keyword in keywordCheck_lst:
        print("Warning: It is better to check whether the extraction is correct manually for file", {fileName})
    
    # match the next section of the risk factor
    nextSectionPattern_lst = [r">item 1b.\s+unresolved staff comments\.?\<", r">item 2.\s+description of property\.?\<"]
    for pattern in nextSectionPattern_lst:
        nextSectionMatch = re.findall(pattern, mainBody.lower())
        
        # if fail to match the pattern, the pattern might be segmented be some html tags, so re-match
        if not nextSectionMatch:
            nextSection = seperated_next_section_rematch(mainBody, pattern)
            # if re-match successfully, jump out of the loop
            if nextSection:
                break
        
        # if match the pattern successfully, just keep the next section keyword and jump out of the loop
        else:
            nextSection = nextSectionMatch[0]
            break
    
    # check if match successfully
    if not nextSection:
        return False
    
    # if match successfully, extract the risk factor
    start_idx = mainBody.lower().find(keyword) # start index is the position of the keyword occured in the main body
    end_idx = mainBody.lower().find(nextSection) # end index is the position of the next section begin in the main body
    riskFactors = mainBody[start_idx: end_idx]
    return riskFactors

In [7]:
def seperated_keyword_rematch(mainBody):
    cleanedMainBody = remove_tags(mainBody)
    keywordMatch = re.findall(r"i\|?t\|?e\|?m\|?\s+\|?1\|?a\|?.\|?\s+\|?r\|?i\|?s\|?k\|?\s+\|?f\|?a\|?c\|?t\|?o\|?r\|?s\.?", cleanedMainBody.lower())
    
    # check if re-match the keyword of risk factor successfully
    if not keywordMatch:
        return False
    
    # if re-match successfully, use the longer segmentation as the keyword of the risk factor
    reMatchedKeyword = max(keywordMatch[0].split('|'), key=len)
    return reMatchedKeyword
    
def seperated_next_section_rematch(mainBody, nextSectionPattern):
    # cleanse the main body text
    cleanedMainBody = remove_tags(mainBody)
    
    # edit the pattern 
    edittedPattern = nextSectionPattern.replace(r">", "").replace(r"\s+", " ").replace(r"\.?\<", "")
    edittedPattern = "\|?".join(edittedPattern)
    edittedPattern = r"{}\.?".format(edittedPattern)
    pattern = edittedPattern.replace(" ", "\s?+")
    nextSectionMatch = re.findall(pattern, cleanedMainBody.lower())
    
    # check if re-match the keyword of next section successfully
    if not nextSectionMatch:
        return False
    
    # if re-match successfully, use the longer segmentation as the keyword of the next section 
    reMatchedNextSection = max(nextSectionMatch[0].split('|'), key=len)
    return reMatchedNextSection

In [8]:
for key, value in filePair_dict.items():
    fileName = key
    fileText = value
    mainBody = separate_main_body(fileText) # separate the original text as the table of content and the main body.
    
    if not mainBody:
        print("The extraction is failed for", {fileName}, "because of the error in seperating main body.")
        continue
    
    riskText = extract_risk_factors(mainBody, fileName) # extract the risk related text
    if not riskText:
        print("The extraction is failed for", {fileName}, "because of the error in extracting risk factors.")
    else:
        riskFilePath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Real Estate Industry/Industry_6500/Raw Text/" + fileName
        with open(riskFilePath, 'w', encoding='utf-8') as file:
            file.write(riskText)

The extraction is failed for {'0000894245-23-000030.txt'} because of the error in seperating main body.
The extraction is failed for {'0000894245-23-000031.txt'} because of the error in seperating main body.
The extraction is failed for {'0000894245-23-000032.txt'} because of the error in seperating main body.
The extraction is failed for {'0000894245-23-000034.txt'} because of the error in seperating main body.
The extraction is failed for {'0000950170-23-007891.txt'} because of the error in seperating main body.
The extraction is failed for {'0000950170-23-011406.txt'} because of the error in seperating main body.
The extraction is failed for {'0000950170-23-011418.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001299969-23-000010.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001408100-23-000055.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001437749-23-007690.txt

### 3. Extract the subjects from raw risk factors text

In [9]:
from bs4 import BeautifulSoup

def extract_subjects(riskFactors, subjectFormat):
    soup = BeautifulSoup(riskFactors, 'html.parser')
    subject_lst = []
    for span in soup.find_all('span'):
        style = span.get('style')
        if style and subjectFormat in style:
            subject_lst.append(span.get_text())
    return subject_lst

In [10]:
rawTextFolderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Real Estate Industry/Industry_6500/Raw Text/"
rawTextFileName_lst = os.listdir(rawTextFolderPath)
rawTextFilePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in rawTextFileName_lst:  
    # encode each document
    filePath = str(rawTextFolderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        rawText = file.read()
    rawTextFilePair_dict[fileName] = rawText

In [11]:
subjectFormat_lst = ["font-weight:700", "font-weight:bold"]
for key, value in rawTextFilePair_dict.items():
    fileName = key
    rawText = value
    
    i = 0
    subject_lst = []
    
    while not subject_lst:
        subjectFormat = subjectFormat_lst[i]
        subject_lst = extract_subjects(rawText, subjectFormat)
        i += 1
        if i >= len(subjectFormat_lst):
            break
        
    if not subject_lst:
        print("The subject fail to extract for", {fileName})
    else:
        subjectFolder = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Real Estate Industry/Industry_6500/Subject/"
        subjectFilePath = subjectFolder + str(fileName)
        with open(subjectFilePath, 'w', encoding='utf-8') as file:
            for item in subject_lst:
                file.write(item + '\n')

The subject fail to extract for {'0000315858-23-000011.txt'}
The subject fail to extract for {'0000844059-23-000014.txt'}
The subject fail to extract for {'0001065949-22-000050.txt'}
The subject fail to extract for {'0001140361-22-034559.txt'}
The subject fail to extract for {'0001213900-23-024126.txt'}
The subject fail to extract for {'0001477932-22-007245.txt'}
The subject fail to extract for {'0001640334-22-001937.txt'}
The subject fail to extract for {'0001654954-23-004006.txt'}


---

## 2.2.2 Industry 6798

### 1. Read in the files and store in a dictionary

In [12]:
folderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Downloaded Financial Reports/Real Estate Industry/Industry_6798/"
fileName_lst = os.listdir(folderPath)
filePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in fileName_lst:  
    # encode each document
    filePath = str(folderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        fileText = file.read()
    filePair_dict[fileName] = fileText

### 2. Extract the raw risk factor text

In [15]:
for key, value in filePair_dict.items():
    fileName = key
    fileText = value
    mainBody = separate_main_body(fileText) # separate the original text as the table of content and the main body.
    
    if not mainBody:
        print("The extraction is failed for", {fileName}, "because of the error in seperating main body.")
        continue
    
    riskText = extract_risk_factors(mainBody, fileName) # extract the risk related text
    if not riskText:
        print("The extraction is failed for", {fileName}, "because of the error in extracting risk factors.")
    else:
        riskFilePath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Real Estate Industry/Industry_6798/Raw Text/" + fileName
        with open(riskFilePath, 'w', encoding='utf-8') as file:
            file.write(riskText)

The extraction is failed for {'0000003499-23-000005.txt'} because of the error in seperating main body.
The extraction is failed for {'0000052827-23-000035.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000104894-23-000028.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000733590-23-000003.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000766704-23-000010.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000790816-23-000013.txt'} because of the error in seperating main body.
The extraction is failed for {'0000803649-23-000030.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000842183-23-000015.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000877860-23-000015.txt'} because of the error in seperating main body.
The extraction is failed for {'0000888491-23-0

The extraction is failed for {'0001558370-23-004185.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001558370-23-004201.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001558370-23-004357.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001558370-23-004524.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001564590-23-002430.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001575311-23-000009.txt'} because of the error in seperating main body.
The extraction is failed for {'0001575965-23-000007.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001628280-23-002955.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001628280-23-003601.txt'} because of the error in extracting risk factors.
The extraction is failed for {'000162828

### 3. Extract the subjects from raw risk factors text

In [16]:
rawTextFolderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Real Estate Industry/Industry_6798/Raw Text/"
rawTextFileName_lst = os.listdir(rawTextFolderPath)
rawTextFilePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in rawTextFileName_lst:  
    # encode each document
    filePath = str(rawTextFolderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        rawText = file.read()
    rawTextFilePair_dict[fileName] = rawText

In [17]:
subjectFormat_lst = ["font-weight:700", "font-weight:bold"]
for key, value in rawTextFilePair_dict.items():
    fileName = key
    rawText = value
    
    i = 0
    subject_lst = []
    
    while not subject_lst:
        subjectFormat = subjectFormat_lst[i]
        subject_lst = extract_subjects(rawText, subjectFormat)
        i += 1
        if i >= len(subjectFormat_lst):
            break
        
    if not subject_lst:
        print("The subject fail to extract for", {fileName})
    else:
        subjectFolder = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Real Estate Industry/Industry_6798/Subject/"
        subjectFilePath = subjectFolder + str(fileName)
        with open(subjectFilePath, 'w', encoding='utf-8') as file:
            for item in subject_lst:
                file.write(item + '\n')

The subject fail to extract for {'0001029800-23-000012.txt'}
The subject fail to extract for {'0001140361-23-010492.txt'}
The subject fail to extract for {'0001171520-23-000122.txt'}
The subject fail to extract for {'0001213900-22-019847.txt'}
The subject fail to extract for {'0001213900-23-018678.txt'}
The subject fail to extract for {'0001387131-23-004110.txt'}
The subject fail to extract for {'0001437749-23-004541.txt'}
The subject fail to extract for {'0001437749-23-007552.txt'}
The subject fail to extract for {'0001437749-23-008235.txt'}
